---
title: "Web Application with Flask"
format:
  html:
    toc: true
author: "Jun Ryu"
date: "2023-02-03"
categories: [application, Flask]
---

# Overview

Today, we will demonstrate how to create a simple web application using `Flask`. 

The application we will develop is a *message bank*. By the end, a user should be able to do two things on the application:

1. **Submit** messages to the bank
2. **View** a random sample of messages stored in the bank

# 1. Enable Submissions
---

First, we will create a base template, `base.html`:

```html
<!doctype html>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
<h1>A Simple Message Bank</h1>
<title>{% block title %}{% endblock %}</title>
<nav>
  <ul>
    <li><a href="{{ url_for('submit') }}">Submit a message</a></li>
    <li><a href="{{ url_for('view') }}">View messages</a></li>
  </ul>
</nav>

<section class="content">
<!-- will be used for submit.html or view.html -->
    
  <header>
    {% block header %}{% endblock %}
  </header>
  {% block content %}{% endblock %}
</section>
```

We will also create `submit.html`, which will `extend` `base.html` and contain the following elements:

1. A text box for submitting a message.
2. A text box for submitting the name of the user.
3. A “submit” button.

Additionally, we will thank the user if a message is submitted.

```html
{% extends 'base.html' %}

{% block header %}
  <h1>{% block title %}Submit{% endblock %}</h1>
{% endblock %}

{% block content %}
  <form method="post">
      <label for="message">Your message: </label>
      <br><br>
      <input type="text" name="message" id="message">
      <br><br>
      <label for="handle">Your name or handle: </label>
      <br><br>
      <input type="text" name="handle" id="handle">
      <br><br>
      <input type="submit" value="Submit message">
  </form>
  
  {% if message %}
  <p>Thank you for the submission!</p>
  {% endif %}
{% endblock %}
```

So far, our combined page will look like this[^1]:

[^1]: Note that the `css` code for the website is attached in the [appendix](#appendix).

![](demo1.png)

---

Now, we will create a new file `app.py`. The first function we will write in this file is `get_message_db()`, which will handle creating the database of messages.

```python
# import correct packages
from flask import Flask, render_template, request
from flask import redirect, url_for, abort, g
import sqlite3
import numpy as np

app = Flask(__name__)

def get_message_db():
    """
    Returns a database in the g attribute of the app with a table to host messages
    """
    
    try:
        return g.message_db # if the database exists
    except:
        g.message_db = sqlite3.connect("messages_db.sqlite")
        # check if a table exists, create one if not 
        cmd = 'CREATE TABLE IF NOT EXISTS messages(id INT, handle TEXT, message TEXT)' 
        cursor = g.message_db.cursor()
        cursor.execute(cmd)
        return g.message_db
```

Next, we write a function `insert_message(request)`, which will handle inserting a user message into the database.

```python
def insert_message(request):
    """
    insert a user message into the database
    """
    db = get_message_db() # connect to database
    cursor = db.cursor()
    cmd = 'SELECT COUNT(*) FROM messages' 
    new_id = cursor.execute(cmd).fetchone()[0] + 1 # set unique id for each message
    # use f-string
    cmd = f"INSERT INTO messages VALUES ({new_id}, '{request.form['handle']}', '{request.form['message']}')"
    cursor.execute(cmd)
    db.commit() # save row insertion
    db.close()
```

Finally, we will write a function `submit()` to `render_template()` the `submit.html` template.

# 2. View Random Submissions
---

fjaklfa

# Appendix

---

Here is the code for `style.css` that was used to customize the app:

```css

body, h1, h2, h3, h4, h5, h6, p, ul, ol, li {
  margin: 0;
  padding: 0;
  font-family: Arial, sans-serif;
}

body {
  background-color: #f5f5f5;
  color: #333;
}

h1 {
  font-size: 32px;
  font-weight: bold;
  margin-bottom: 20px;
  color: #007bff;
  text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);
  text-align: center;
}

nav {
  background-color: #333;
  color: #fff;
  padding: 10px;
}

nav ul {
  list-style-type: none;
  text-align: center;
}

nav li {
  display: inline-block;
  margin-right: 10px;
}

nav a {
  color: #fff;
  text-decoration: none;
}

nav a:hover {
  text-decoration: underline;
}

.content {
  margin: 20px;
}

header {
  text-align: center;
  margin-bottom: 20px;
}

.block {
  background-color: #fff;
  padding: 20px;
  border-radius: 4px;
  box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
}

.submit-link,
.view-link {
  display: inline-block;
  padding: 10px 20px;
  background-color: #007bff;
  color: #fff;
  border: none;
  border-radius: 4px;
  cursor: pointer;
  transition: background-color 0.3s;
  text-decoration: none;
}

.submit-link:hover,
.view-link:hover {
  background-color: #0056b3;
}
```